## Song lyric puller

I'm going to analyze the songs in my music library.

I've been using Google Play Music (GPM) for about 10 years now. I have all the albums I own uploaded to my digital library, I've made playlists for friends and family and myself, and collected all of the playlists of the radio show I used to host in grad school. Recent actions by Google have foreshadowed the shuddering of this service. They have a near identical service, Youtube Music, running in parallel to GPM. They appear to have decided Youtube Music will be there music streaming service and have been phasing GPM out. This puts my music and my playlists in jeoparady. So, I found a way to extract all that info to CSV files.

Now, I'd like to do some Natural Language Processing on this lists to glean some interesting info out of them. First however, I think if we are dealing with songs and NLP, we should collect the lyrics to those songs too.

This Webscraping routine pulls the lyrics of each of the songs in my library from Genius using selenium, parses it using Beautiful Soup, and adds it my master GPM list of songs using pandas.

In [2]:
#bring in all the libraries I'll need

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time
import string
import requests
from bs4 import BeautifulSoup

import unidecode
import pandas as pd

In [3]:
# set driver options and import song list
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')

gpm=pd.read_csv('GPM_song_list.csv')

gpm.head(5)

,artist,album,title,duration,playcount,rating,rating_interpretation
0,Los Campesinos!,"Hold On Now, Youngster",... And We Exhale And Roll Our Eyes In Unison,2:49,1.0,0,not-rated
1,The Steinways,Missed The Boat,... But You're Really Nice,0:17,2.0,undefined,never-rated
2,Titus Andronicus,The Most Lamentable Tragedy,[ intermission ],1:17,1.0,undefined,never-rated
3,A Giant Dog,Pile,& Rock & Roll,2:56,36.0,5,thumbs-up
4,Titus Andronicus,The Most Lamentable Tragedy,[ seven seconds ],0:07,NaN,undefined,never-rated


In [50]:
#set up driver object and collect lyrics

driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.implicitly_wait(5)

#pull band and song info needed to direct to correct webpage on genius and format it correctly, remove punctuation, 
#replace spaces with hyphons.

i=1200

band=gpm.loc[i][0]
band = unidecode.unidecode(band)
band=band.replace('&','and')
band=band.replace(' ','-')
band=band.replace('!','')
song=gpm.loc[i][2]
song = unidecode.unidecode(song)
song=song.replace('&','and')
song=song.replace('...','')
song=song.replace(' ','-')

try:
    driver.get(f'https://genius.com/{band}-{song}-lyrics')
    
    #pull lyrics from page

    page= driver.find_element_by_xpath(f'/html/body/routable-page/ng-outlet/song-page/div/div/div/div/div')

    soup=BeautifulSoup(page.text,'html.parser')

    #clean lyrics by changing soup object to list of strings, removing \n  and \u2005 characters, and non lyric strings
    #at the beginning and end of each page.

    s=str(soup)
    s=s.replace('\u2005',' ')
    s=s.split('\n')
    s.pop()
    s.pop()
    s.pop(0)

    # remove blank strings and strings containing song structure info.

    s=[x for x in s if '[' not in x]
    for line in s:
        if line == '':
            s.remove('')

    # concatenate the list of strings into a single string. 
    lyrics=''
    for line in s:        
        lyrics+=str(line)
        lyrics+=' '
    print(lyrics)

except:
    print('web page did not resolve')

finally:
    driver.quit()

web page did not resolve


The routine works for a single song, now write a loop that will iterate through the entire library. This is a slow process, and once in a while (with how tempormental Webdrivers tend to be) the program crashes. It continues to iterate through the loop, but does not continue to access webpages correctly.

In [4]:
lst=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for e in lst:
    print(e*913)
    
print(len(gpm.index))

913
1826
2739
3652
4565
5478
6391
7304
8217
9130
10043
10956
11869
12782
13695
14608
15521
16434
17347
18260
18266


In [ ]:
RAW_1=[]
URL_1=[]
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

#pull band and song info needed to direct to correct webpage on genius and format it correctly
error = 0
for i in range(0,914):
    
    band=gpm.loc[i][0]
    
    band=band.replace('&','and')
    band=band.replace(' ','-')
    band=band.replace('!','')
    band=band.replace('?','')
    band=band.replace(',','')
    band=band.replace("'",'')
    band=band.replace('"','')
    band=band.replace(':','')
    band=band.replace(';','')
    band = unidecode.unidecode(band)
    
    song=gpm.loc[i][2]
    
    song=song.replace('&','and')
    song=song.replace('...','')
    song=song.replace('(','')
    song=song.replace(')','')
    song=song.replace(' ','-')
    song=song.replace(',','')
    song=song.replace("'",'')
    song=song.replace('"','')
    song=song.replace(':','')
    song=song.replace(';','')
    song = unidecode.unidecode(song)

    url=f'https://genius.com/{band}-{song}-lyrics'
    
    try:
        #access page
        
        time.sleep(2)
        driver.get(url)
        
        #pull lyrics from page
        
        time.sleep(2)
        page=driver.find_element_by_class_name('lyrics')
        soup=BeautifulSoup(page.text,'html.parser')

        #clean lyrics by changing soup object to list of strings, removing \n  and \u2005 characters, and non lyric strings
        #at the beginning and end of each page.

        s=str(soup)
        s=s.replace('\u2005',' ')
        s=s.split('\n')
        s.pop()
        s.pop()
        s.pop(0)

        # remove blank strings and concatenate the list of strings into a single string.

        s=[x for x in s if '[' not in x]
        for line in s:
            if line == '':
                s.remove('')
            elif line== 'SEE MORE':
                s.remove('SEE MORE')

        # concatenate the list of strings into a single string. 
        lyrics=''
        for line in s:        
            lyrics+=str(line)
            lyrics+=' '
        #print(lyrics)
        RAW_1.append(lyrics)
        URL_1.append(url)
        print(i)
    except:
        error+=1
        print(f'Failed. url used was {url} error number {error}')
        lyrics='F'
        RAW_1.append(lyrics)
        URL_1.append(url)
        pass
        

print('done')
driver.quit()

Failed. url used was https://genius.com/Los-Campesinos--And-We-Exhale-And-Roll-Our-Eyes-In-Unison-lyrics error number 1
Failed. url used was https://genius.com/The-Steinways--But-Youre-Really-Nice-lyrics error number 2
Failed. url used was https://genius.com/Titus-Andronicus-[-intermission-]-lyrics error number 3
Failed. url used was https://genius.com/A-Giant-Dog-and-Rock-and-Roll-lyrics error number 4
Failed. url used was https://genius.com/Titus-Andronicus-[-seven-seconds-]-lyrics error number 5
Failed. url used was https://genius.com/M83-*-lyrics error number 6
Failed. url used was https://genius.com/Hop-Along-Queen-Ansleis---lyrics error number 7
Failed. url used was https://genius.com/Zt--------------lyrics error number 8
Failed. url used was https://genius.com/Ceremony-/-lyrics error number 9
Failed. url used was https://genius.com/Ceremony-//-lyrics error number 10
Failed. url used was https://genius.com/Ceremony-///-lyrics error number 11
Failed. url used was https://genius.co

143
144
Failed. url used was https://genius.com/Origami-Angel-24-Hr-Delivery-/-KD-MVP-lyrics error number 88
146
Failed. url used was https://genius.com/Oh-Rose-25-Alive-lyrics error number 89
148
Failed. url used was https://genius.com/Bon-Iver-29-#Strafford-APTS-lyrics error number 90
150
Failed. url used was https://genius.com/Ted-Leo-and-The-Pharmacists-2nd-Ave-11-A.M.-lyrics error number 91
Failed. url used was https://genius.com/Cloud-District-2sad2swim-lyrics error number 92
Failed. url used was https://genius.com/Pllush-345-lyrics error number 93
154
155
156
Failed. url used was https://genius.com/Caves-<3-Koala-lyrics error number 94
158
Failed. url used was https://genius.com/Superchunk-30-Xtra-lyrics error number 95
Failed. url used was https://genius.com/Lightning-Bolt-30000-Monkies-lyrics error number 96
Failed. url used was https://genius.com/Dannys-Dead-319-lyrics error number 97
Failed. url used was https://genius.com/Bon-Iver-33-"GOD"-lyrics error number 98
163
164
165

ran a test of 100 songs. It was necessary to tell the program to wait a few seconds for the page to load. I always find tell python on the command line to hold tends to work better with selenium. When trying to use seleniums more advanced wait actions it seems to fall over itself and crash. 

after 100 songs, I had 38 lyrics webpages that didn't resolve. I'd like them to all run but they fail for various reasons. I had the output of the routine print the url that was attempted to help troubleshoot why some of the urls didn't work. The colprite for many was commas and apostrophes in the song title that were not in the url. run the 100 song test again, removing them before being entered.

correcting the punctuation cut down the errors from 38 to 28. The rest are more unique issues that would take to long to address each issue for each song. The other situation is that genius doesn't carry the songs lyrics at all. 72% of 18000+ songs will still be plenty of text to do analysis on. 

now exporting the url and lyric information to a set of two lists, keeping track of the url used and will add them to the GPM dataframe when they are finished.

In [1]:
RAW_1

NameError: name 'RAW_1' is not defined